In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import required libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../input/automobile-dataset/Automobile_data.csv')
data.head(6)

In [ ]:
data.info()

In [ ]:
for col in data.columns:
    for i in range(205):
        if data.loc[i,col] == '?':
            data.loc[i,col] = np.nan

In [ ]:
data.isna().sum()

In [ ]:
#change numinal datas which were seted wrongly object
false_obj = ['bore','stroke','horsepower','peak-rpm','price']
data[false_obj] = data[false_obj].astype('float64')

In [ ]:
knn_impute = KNNImputer()
#define numinak columns
num_col = [col for col in data.columns if data[col].dtype in ['int64','float64']]
#fill nan data in numinal columns
data_num = pd.DataFrame(knn_impute.fit_transform(data[num_col]))
#return numinal columns name to data_num
data_num.columns = data[num_col].columns
#drop num_col from original dataset
data = data.drop(num_col,axis=1)
#concat imputed numinal dataset to main dataset
data = pd.concat([data,data_num],axis=1)

In [ ]:
label =data.price
data = data.drop(['price','normalized-losses'],axis=1)

In [ ]:
data.info()

In [ ]:
corr_lst = [col for col in data.columns if data[col].dtype=='float64']
corr = pd.concat([data[corr_lst],label],axis=1)
fig = plt.figure(figsize=(12,10),dpi=80)
sns.heatmap(corr.corr().abs(),annot=True);

In [ ]:
sns.set_theme(style="whitegrid")
obj_lst = data.drop(corr_lst,axis=1).columns
def bar_plt(feature,label):
    sns.barplot(x=data[feature],y=label,estimator=np.mean)
for f in obj_lst:
    fig = plt.figure(figsize=(10,6),dpi=70)
    bar_plt(f,label)
    plt.xticks(rotation = 90)

In [ ]:
# categorize make column
categ_df = pd.concat([data[obj_lst],label],axis=1)
categ_df["make"] = (categ_df.groupby("make")["price"].transform("mean"))
#as you can see in the obove graph, the number of doors almost has no effrct on the price of the car, so I drpped its column
categ_df = categ_df.drop(['price','num-of-doors'],axis=1)

for i in categ_df.make:
    if i>25000:
        categ_df.make[(list(categ_df.make)).index(i)] = int(2)
    elif i<25000 and i>15000:
        categ_df.make[(list(categ_df.make)).index(i)]  = int(1)
    else:
        categ_df.make[(list(categ_df.make)).index(i)] = int(0)

In [ ]:
# categorize engine column
categ_df['engine-type'] = [1 if i in ['dohc', 'dohcv','ohcv'] else 0 for i in categ_df['engine-type']]

In [ ]:
# categorize cylinders column
categ_df['num-of-cylinders'] = [0 if i in ['two', 'three','four'] else 1 for i in categ_df['num-of-cylinders']]

In [ ]:
cat_lst = [col for col in categ_df.columns if len(set(categ_df[col]))<4]
new_data = pd.concat([categ_df[cat_lst],data[corr_lst]],axis=1)

In [ ]:
new_data = pd.get_dummies(new_data,drop_first=True)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(new_data,label,random_state=42)

In [ ]:
#normalize
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [ ]:
#pca
pca = PCA(20)
X_train_pca = pca.fit_transform(X_train_scaled)
for i in range(15):
    print(f'PCA {i+1} has {np.cumsum(pca.explained_variance_ratio_*100)[i]}')

In [ ]:
fig = plt.figure(figsize=(10,8),dpi=90)
sns.heatmap(np.corrcoef(X_train_pca.T))

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()),
 ('pca', PCA(20)),
 ('model', XGBRegressor(objective='reg:squarederror'))])
pipe.fit(X_train,y_train)
pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error,r2_score
r2_score = r2_score(y_test,pred)
print('R2 score = ',r2_score)
error = mean_absolute_percentage_error(y_test,pred)
print('MAPE = ',error)

In [ ]:
prediction = pd.DataFrame(pred)
ground_truth = pd.DataFrame(y_test.to_list())
prediction.columns=['Prediction']
ground_truth.columns=['True value']
compare_df = pd.concat([prediction,ground_truth],axis=1)

In [ ]:
# here you can see the real price and predicted price together
compare_df

In [ ]:
# comparing the pred and true value alongside each other on bar plot
labels = [i for i in range(30)]
pred = compare_df.iloc[:30,0]
true = compare_df.iloc[:30,1]
x = np.arange(len(labels))
width = 0.3
fig = plt.figure(figsize=(14,10),dpi=80)
rects1 = plt.bar(x - width/2, pred, width, label='Prediction')
rects2 = plt.bar(x + width/2, true, width, label='True value')
plt.legend()
plt.tight_layout()
plt.show()